In [2]:
#git@github.com:analyticsrepo01/gemini_video.git

In [12]:
!pip install google-cloud-videointelligence
!pip install pytube
!pip install unidecode
!pip install varname

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 kB 1.7 MB/s eta 0:00:0000:010:01
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
Using cached pytube-15.0.0-py3-none-any.whl (57 kB)


In [23]:
# %pip install google-cloud-videointelligence
# %pip install unidecode
# %pip install varname
%pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 2.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
"""Transcribe speech from a video stored on GCS."""
from google.cloud import videointelligence
from pytube import YouTube
# Install pytube if not already installed
from pytube import YouTube
from google.cloud import storage
import vertexai
import json
import unidecode
from varname import nameof

import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)
from vertexai.generative_models import GenerativeModel

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason

### Setup

In [25]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [26]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    # model_name="gemini-pro", # Replace with your desired Gemini model
    # model = GenerativeModel("gemini-pro")
    # model = GenerativeModel("gemini-1.5-pro-preview-0215")
    model = "gemini-1.5-pro-preview-0409",
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


def generate_pro(input_prompt):
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    full_prompt = '''summarize the prompt below and do note prompt below will be send to imagen mode so please clean up any sensitve words and replace them into unblocked words like replace girl or lady can be replaced by female human and so on : ''' + input_prompt
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 8190,
        "temperature": 0.2,
        "top_p": 1
    },stream=False,)
    
    # print (responses.text)
    
    return(responses.text)



In [42]:
def generate(input_array, max_output_tokens=8192, temperature=1, top_p=0.95):
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    params = {}
    params["safety_settings"] = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }
    params["generation_config"] = {"max_output_tokens": max_output_tokens,"temperature": temperature,"top_p": top_p}
    params["stream"] = True
    ##responses = model.generate_content(input_array,generation_config=generation_config,safety_settings=safety_settings, stream=True)
    responses = model.generate_content(input_array,**params)
    return responses

In [72]:
from google.cloud import storage
# -1) Helper functions for bucket management
# -1.1) Fetching from youtube
def download_video(video_url):
    # Create a YouTube object
    yt = YouTube(video_url)
    
    # Filter to get the highest resolution stream that includes both video and audio
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    
    # Download the video
    if stream:
        print(f"Going to download {video_url}")
        stream.download()
        print(f"Downloaded into {stream.default_filename}")
    else:
        print("No suitable stream found")
    return stream.default_filename
def upload_video_sur_bucket(video_url):
    downloaded_video     = download_video(video_url)
    uploaded_video       = upload_file_to_bucket(bucket_name, downloaded_video)
    
    uri_video_sur_bucket = f"gs://{bucket_name}/{uploaded_video}"

    return uri_video_sur_bucket
# -1.2) Managing buckets
def list_buckets_names():
    storage_client = storage.Client()
    buckets        = storage_client.list_buckets()
    return [bucket.name for bucket in buckets]


def create_bucket_class_location(bucket_name, storage_class = "STANDARD", location="us"):
    storage_client       = storage.Client()
    bucket               = storage_client.bucket(bucket_name)
    bucket.storage_class = storage_class
    new_bucket           = storage_client.create_bucket(bucket, location=location)
    print(f"Created bucket {new_bucket.name} in {new_bucket.location} with storage class {new_bucket.storage_class}")
    return new_bucket


def upload_file_to_gcs(bucket_name, source_file_path, destination_blob_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)
    return blob
def upload_file_to_bucket(bucket_name, source_file_path_name): 
    source_file_path = f"./{source_file_path_name}"
    destination_blob_name = source_file_path_name.replace(' ', '_')
    destination_blob_name = unidecode.unidecode(destination_blob_name)
    uploaded_blob         = upload_file_to_gcs(bucket_name, source_file_path, destination_blob_name)
    print(f"File [{source_file_path}] \nUploaded to [{uploaded_blob.name}] \nIn bucket [{bucket_name}]")
    return destination_blob_name
def create_bucket_if_doesnt_exist(bucket_name):
    if bucket_name not in list_buckets_names():
        print(f"Going to create bucket :{bucket_name}")
        create_bucket_class_location(bucket_name)
        
    assert(bucket_name in list_buckets_names())

    
# - 1.3) 🔥 Calling Gemini 🔥 (the magic is here ! )
def generate(input_array, max_output_tokens=8192, temperature=1, top_p=0.95):
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    params = {}
    params["safety_settings"] = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }
    params["generation_config"] = {"max_output_tokens": max_output_tokens,"temperature": temperature,"top_p": top_p}
    params["stream"] = True
    ##responses = model.generate_content(input_array,generation_config=generation_config,safety_settings=safety_settings, stream=True)
    responses = model.generate_content(input_array,**params)
    return responses

In [73]:
# Define the video URL
video_url = "https://www.youtube.com/watch?v=BZ9SaUIUL58"


# Set with your own parameters
bucket_name = "my-project-0004-bucket"

In [74]:
vertexai.init(project=PROJECT_ID, location="us-central1")
create_bucket_if_doesnt_exist(bucket_name)

In [75]:
# 1.2) transfer youtube video to the bucket

downloaded_video = download_video(video_url)
uploaded_video = upload_file_to_bucket(bucket_name, downloaded_video)

Going to download https://www.youtube.com/watch?v=BZ9SaUIUL58
Downloaded into Tried out Googles Code Gemma.mp4


FileNotFoundError: [Errno 2] No such file or directory: '/Tried out Googles Code Gemma.mp4'

In [71]:
uploaded_video

'./Tried_out_Googles_Code_Gemma.mp4'

### 1.3) Add the video as a Part of the model


In [35]:
from vertexai.generative_models import GenerativeModel

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models


In [53]:
import os

filename = uploaded_video
base_name = os.path.basename(filename)  # Get the base filename
# name_without_ext = os.path.splitext(base_name)[0]  # Split off the extension 

print(base_name) 

Budget_participatif_Saison_3___votez_jusquau_11_juin_!.mp4


In [54]:
uri_video_sur_bucket = f"gs://{bucket_name}/{base_name}"
mime_type = "video/mp4"
parameters = {"mime_type"  : mime_type, 
              "uri"        : uri_video_sur_bucket }
video = Part.from_uri(**parameters)

In [64]:
prompt = """"Create a chapter index for this video and generate a nice HTML code to present the chapter index."""
responses = generate([prompt, video])

In [65]:
result = []
for response in responses :
    print(".",end="")
    result.append(response.text)

................

In [66]:
print("\n".join(result))


##
  Budget Participatif - Suresnes - Saison 3 - Index des Cha
pitres

Malheureusement, je ne peux pas créer un index des chapitres
 complet ni générer le code HTML correspondant sans plus d'informations sur le contenu de la vidéo.  Pour créer un index précis, j'aurais besoin de détails
 sur les sujets abordés et les moments clés de la vidéo. 

Cependant, je peux vous fournir un exemple de structure d'index et de code HTML
 que vous pouvez adapter à votre vidéo :

**Exemple de structure d'index :**

* **Introduction (00:00)**
* **Qu'est-ce que le budget participatif ? (00:1
5)**
* **Projets de la saison 2 (00:30)**
    * **Compostage (00:40)**
    * **Chiens en EHPAD (00:5
0)**
    * **Nichoirs à oiseaux (01:00)**
    * **Jardin partagé (01:10)**
* **Projets de la saison 3 (01:20)**
    * **Tables de pique-nique (01:30
)**
    * **Casques audiodescription (01:40)**
    * **Ateliers du rire pour séniors (01:50)**
* **Comment voter ? (02:00)**
* **Date limite de vote (02:10)**
*
 **Concl